In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sympy import *
from robot_tools import *
import math

In [10]:
t = symbols('t')
theta1 = Function(r"\theta_1")(t)
theta1_dot = diff(theta1, t)
theta1_ddot = diff(theta1, t, t)

theta2 = Function(r"\theta_2")(t)
theta2_dot = diff(theta2, t)
theta2_ddot = diff(theta2, t, t)

l1 = symbols('L1')
l2 = symbols('L2')
d3 = symbols('d3')

alias = {}

# display as theta dot instead of d(theta)/dt, and ignore dependency (t)
alias.update({theta1: symbols(r"\theta_1"),
              theta1_dot: symbols(r"\dot{\theta_1}"),
              theta1_ddot: symbols(r"\ddot{\theta_1}"),
              theta2: symbols(r"\theta_2"),
              theta2_dot: symbols(r"\dot{\theta_2}"),
              theta2_ddot: symbols(r"\ddot{\theta_2}")
             })

In [11]:
# Q1

DH1 = [0, 0, l1, theta1]
T01 = Tlink(DH1)
symprint('T', 0, 1)
matprint(T01, alias)

DH2 = [0, -pi/2, l2, theta2]
T12 = Tlink(DH2)
symprint('T', 1, 2)
matprint(T12, alias)

DH3 = [0, -pi/2, d3, 0]
T23 = Tlink(DH3)
symprint('T', 2, 3)
matprint(T23, alias)


T03 = simplify(T01 * T12 * T23)
symprint('T', 0, 3)
matprint(T03, alias)


^0T_1

Matrix([
[cos(\theta_1), -sin(\theta_1), 0,  0],
[sin(\theta_1),  cos(\theta_1), 0,  0],
[            0,              0, 1, L1],
[            0,              0, 0,  1]])

^1T_2

Matrix([
[ cos(\theta_2), -sin(\theta_2), 0,  0],
[             0,              0, 1, L2],
[-sin(\theta_2), -cos(\theta_2), 0,  0],
[             0,              0, 0,  1]])

^2T_3

Matrix([
[1,  0, 0,  0],
[0,  0, 1, d3],
[0, -1, 0,  0],
[0,  0, 0,  1]])

^0T_3

Matrix([
[cos(\theta_1)*cos(\theta_2),  sin(\theta_1), -sin(\theta_2)*cos(\theta_1), -L2*sin(\theta_1) - d3*sin(\theta_2)*cos(\theta_1)],
[sin(\theta_1)*cos(\theta_2), -cos(\theta_1), -sin(\theta_1)*sin(\theta_2),  L2*cos(\theta_1) - d3*sin(\theta_1)*sin(\theta_2)],
[             -sin(\theta_2),              0,               -cos(\theta_2),                              L1 - d3*cos(\theta_2)],
[                          0,              0,                            0,                                                  1]])